In [123]:
def pitch_classification(mlbid, output):
    #importing required libraries
    import pandas as pd
    import lightgbm as lgb
    import numpy as np
    from pybaseball import statcast_pitcher, playerid_lookup
    from sklearn.metrics import precision_score, confusion_matrix, classification_report
    from sklearn.model_selection import train_test_split
    
    #gathering pitch-by-pitch data for the pitcher
    data = statcast_pitcher('2019-03-15','2019-11-30',mlbid)
    
    #removes all pitches that are intentional balls or pitchouts
    data = data.loc[(data['description']!= 'intent_ball') & (data['pitch_type']!='PO')]
    
    #getting rid of any pitch types that aren't correctly classified
    data = data.loc[data['pitch_type'].isnull() == False]
    
    new_data = data.loc[(data['release_pos_x'].isnull()==False) & 
                            (data['release_pos_z'].isnull()==False)]
    
    #getting all pitch types for the pitcher
    pitches = new_data.pitch_type.unique().tolist()
    
    #this loop removes pitch types that aren't thrown more than 7.5% of the time to make classifying more accurate
    for i in range(len(pitches)):
        if (len(new_data[new_data['pitch_type']== pitches[i]])/len(new_data)) <= 0.075:
            new_data = new_data[new_data['pitch_type']!= pitches[i]]
        
    #defining new pitch list if some pitch types were dropped in the loop above
    pitches = new_data.pitch_type.unique().tolist()
    
    #converting pitche types from strings to integers
    new_data['pitch_type'].replace(pitches, list(range(len(pitches))),inplace = True)
    new_data['pitch_type'] = new_data['pitch_type'].apply(pd.to_numeric)
    
    #getting the necessary columns for the model
    X1 = new_data[['release_pos_x', 'release_pos_z', 'strikes','balls']]

    #singling out the variable we want to predict
    Y1 = new_data['pitch_type']
    
    #splitting the data into training and testing sets
    X_train,X_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.3,random_state=0)
    
    d_train=lgb.Dataset(X_train, label=y_train)
    
    #setting up the parameters
    params={}
    params['num_leaves'] = 10
    params['learning_rate']=0.03
    params['boosting_type']='gbdt' #GradientBoostingDecisionTree
    params['objective']='multiclass' #Multi-class target feature
    params['metric']='multi_logloss' #metric for multi-class
    params['max_depth']= 5
    params['num_class']= len(pitches) #no.of unique values in the target class not inclusive of the end value
    params['min_data_in_leaf'] = 10
    params['class_weight'] = 'balanced'
    
    #training the model
    clf=lgb.train(params,d_train,500)
    
    #prediction on the test dataset
    y_pred_1=clf.predict(X_test)
    
    #argmax() method 
    y_pred_1 = [np.argmax(line) for line in y_pred_1]
    
    #using precision score for error metrics
    score = precision_score(y_pred_1,y_test,average=None).mean()
    
    matrix = confusion_matrix(y_test, y_pred_1)
    
    report = classification_report(y_test, y_pred_1)
    
    if output == 'score':
        return score
    
    elif output == 'matrix':
        return matrix
    
    elif output == 'report':
        return report
    else:
        return score

In [25]:
from pybaseball import playerid_lookup

In [108]:
playerid_lookup('verlander','justin')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,verlander,justin,434378,verlj001,verlaju01,8700,2005.0,2020.0


In [109]:
pitch_classification(434378, 'score')

Gathering Player Data


0.5382811821755901

In [138]:
playerid_lookup('castillo','luis')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,castillo,luis,518088,NaN,NaN,-1,NaN,NaN
1,castillo,luis,112116,castl001,castilu01,513,1996.0,2010.0
2,castillo,luis,514630,NaN,NaN,-1,NaN,NaN
3,castillo,luis,642278,NaN,NaN,-1,NaN,NaN
4,castillo,luis,570491,NaN,NaN,-1,NaN,NaN
5,castillo,luis,471039,NaN,NaN,-1,NaN,NaN
6,castillo,luis,-1,NaN,NaN,-1,NaN,NaN
7,castillo,luis,622379,NaN,NaN,-1,NaN,NaN
8,castillo,luis,683463,NaN,NaN,-1,NaN,NaN
9,castillo,luis,503082,NaN,NaN,-1,NaN,NaN


In [139]:
pitch_classification(622491, 'score')

Gathering Player Data


0.5178694932553993

In [150]:
from pybaseball import pitching_stats_bref

In [151]:
season_stats = pitching_stats(2019)

In [152]:
pd.set_option('display.max_columns',None)

In [153]:
season_stats

,Season,Name,Team,Age,W,L,ERA,WAR,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,Strikes,Pitches,RS,IFH,BU,BUH,K/9,BB/9,K/BB,H/9,HR/9,AVG,WHIP,BABIP,LOB%,FIP,GB/FB,LD%,GB%,IFFB%,HR/FB,IFH%,BUH%,Starting,Start-IP,Relieving,Relief-IP,RAR,Dollars,tERA,xFIP,WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,Clutch,FB%,FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,HLD,SD,MD,ERA-,FIP-,xFIP-,K%,BB%,SIERA,RS/9,E-F,FA% (pfx),FT% (pfx),FC% (pfx),FS% (pfx),FO% (pfx),SI% (pfx),SL% (pfx),CU% (pfx),KC% (pfx),EP% (pfx),CH% (pfx),SC% (pfx),KN% (pfx),UN% (pfx),vFA (pfx),vFT (pfx),vFC (pfx),vFS (pfx),vFO (pfx),vSI (pfx),vSL (pfx),vCU (pfx),vKC (pfx),vEP (pfx),vCH (pfx),vSC (pfx),vKN (pfx),FA-X (pfx),FT-X (pfx),FC-X (pfx),FS-X (pfx),FO-X (pfx),SI-X (pfx),SL-X (pfx),CU-X (pfx),KC-X (pfx),EP-X (pfx),CH-X (pfx),SC-X (pfx),KN-X (pfx),FA-Z (pfx),FT-Z (pfx),FC-Z (pfx),FS-Z (pfx),FO-Z (pfx),SI-Z (pfx),SL-Z (pfx),CU-Z (pfx),KC-Z (pfx),EP-Z (pfx),CH-Z (pfx),SC-Z (pfx),KN-Z (pfx),wFA (pfx),wFT (pfx),wFC (pfx),wFS (pfx),wFO (pfx),wSI (pfx),wSL (pfx),wCU (pfx),wKC (pfx),wEP (pfx),wCH (pfx),wSC (pfx),wKN (pfx),wFA/C (pfx),wFT/C (pfx),wFC/C (pfx),wFS/C (pfx),wFO/C (pfx),wSI/C (pfx),wSL/C (pfx),wCU/C (pfx),wKC/C (pfx),wEP/C (pfx),wCH/C (pfx),wSC/C (pfx),wKN/C (pfx),O-Swing% (pfx),Z-Swing% (pfx),Swing% (pfx),O-Contact% (pfx),Z-Contact% (pfx),Contact% (pfx),Zone% (pfx),Pace,RA9-WAR,BIP-Wins,LOB-Wins,FDP-Wins,Age Rng,K-BB%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,kwERA,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi)
94,2019.0,Gerrit Cole,Astros,28.0,20.0,5.0,2.50,7.4,33.0,33.0,0.0,0.0,0.0,0.0,212.1,817.0,142.0,66.0,59.0,29.0,48.0,0.0,3.0,4.0,3.0,326.0,176.0,172.0,89.0,19.0,1108.0,2254.0,3362.0,177.0,11.0,3.0,1.0,13.82,2.03,6.79,6.02,1.23,0.185,0.89,0.275,0.833,2.64,1.02,0.204,0.403,0.110,0.169,0.063,0.333,69.7,212.1,NaN,NaN,69.7,$59.3,2.59,2.48,4.31,-10.41,14.72,49.91,4.94,0.90,0.89,0.88,0.88,33.0,5.13,-0.34,0.540,97.2,0.232,89.3,NaN,NaN,0.154,82.6,0.074,88.6,NaN,NaN,NaN,NaN,NaN,NaN,37.1,13.6,NaN,0.0,0.4,NaN,NaN,2.05,1.75,NaN,0.00,0.16,NaN,NaN,0.350,0.679,0.499,0.491,0.771,0.663,0.452,0.641,0.168,0.0,0.0,0.0,56.0,59.0,55.0,0.399,0.059,2.62,7.50,-0.14,0.516,0.024,NaN,NaN,NaN,NaN,0.232,NaN,0.155,NaN,0.074,NaN,NaN,NaN,97.1,97.1,NaN,NaN,NaN,NaN,89.2,NaN,82.6,NaN,88.6,NaN,NaN,-7.2,-9.2,NaN,NaN,NaN,NaN,2.0,NaN,5.9,NaN,-9.3,NaN,NaN,10.7,9.0,NaN,NaN,NaN,NaN,2.0,NaN,-7.8,NaN,7.2,NaN,NaN,21.3,-0.2,NaN,NaN,NaN,NaN,23.2,NaN,8.3,NaN,4.1,NaN,NaN,1.23,-0.29,NaN,NaN,NaN,NaN,2.98,NaN,1.60,NaN,1.65,NaN,NaN,0.333,0.650,0.498,0.450,0.763,0.662,0.521,22.7,7.8,0.9,-0.6,0.3,28 - 28,0.340,0.373,0.343,0.284,0.193,0.468,0.339,2.15,0.493,0.074,NaN,0.155,0.536,NaN,NaN,NaN,NaN,0.004,0.231,NaN,88.9,NaN,82.8,97.4,NaN,NaN,NaN,NaN,96.0,89.5,NaN,-8.8,NaN,6.7,-6.5,NaN,NaN,NaN,NaN,-9.0,2.9,NaN,6.0,NaN,-9.1,9.9,NaN,NaN,NaN,NaN,7.4,1.0,NaN,0.5,NaN,0.2,34.4,NaN,NaN,NaN,NaN,0.1,14.6,NaN,0.18,NaN,0.03,1.91,NaN,NaN,NaN,NaN,0.96,1.88,NaN,0.339,0.638,0.498,0.447,0.763,0.662,0.530,22.9
85,2019.0,Jacob deGrom,Mets,31.0,11.0,8.0,2.43,7.0,32.0,32.0,0.0,0.0,0.0,0.0,204.0,804.0,154.0,59.0,55.0,19.0,44.0,1.0,7.0,2.0,0.0,255.0,217.0,169.0,1

In [199]:
#the rest of the notebook is me calculating the precision scores for 16 players and adding the data to a df
playerid_lookup('Scherzer', 'Max')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,scherzer,max,453286,schem001,scherma01,3137,2008.0,2020.0


In [200]:
ms_score = pitch_classification(453286, 'score')

Gathering Player Data


In [249]:
names = []
prec_scores = []
adj_scores = []

In [250]:
names.append('Max Scherzer')
prec_scores.append(ms_score)
adj_scores.append(ms_score/0.25)

In [203]:
playerid_lookup('cole', 'gerrit')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,cole,gerrit,543037,coleg001,colege01,13125,2013.0,2020.0


In [204]:
gc_score = pitch_classification(543037, 'score')

Gathering Player Data


In [251]:
names.append('Gerrit Cole')
prec_scores.append(gc_score)
adj_scores.append(gc_score/(1/3))

In [206]:
playerid_lookup('glasnow','tyler')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,glasnow,tyler,607192,glast001,glasnty01,14374,2016.0,2020.0


In [207]:
tg_score = pitch_classification(607192, 'score')

Gathering Player Data


In [252]:
names.append('Tyler Glasnow')
prec_scores.append(tg_score)
adj_scores.append(tg_score/0.5)

In [209]:
playerid_lookup('bumgarner','madison')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,bumgarner,madison,518516,bumgm001,bumgama01,5524,2009.0,2020.0


In [210]:
mb_score = pitch_classification(518516, 'score')

Gathering Player Data


In [253]:
names.append('Madison Bumgarner')
prec_scores.append(mb_score)
adj_scores.append(mb_score/(1/3))

In [212]:
playerid_lookup('paddack','chris')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,paddack,chris,663978,paddc001,paddach01,20099,2019.0,2020.0


In [213]:
cp_score = pitch_classification(663978, 'score')

Gathering Player Data


In [254]:
names.append('Chris Paddack')
prec_scores.append(cp_score)
adj_scores.append(cp_score/(1/3))

In [215]:
playerid_lookup('degrom','jacob')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,degrom,jacob,594798,degrj001,degroja01,10954,2014.0,2020.0


In [216]:
jd_score = pitch_classification(594798, 'score')

Gathering Player Data


In [255]:
names.append('Jacob DeGrom')
prec_scores.append(jd_score)
adj_scores.append(jd_score/(1/3))

In [219]:
playerid_lookup('verlander','justin')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,verlander,justin,434378,verlj001,verlaju01,8700,2005.0,2020.0


In [220]:
jv_score = pitch_classification(434378, 'score')

Gathering Player Data


In [256]:
names.append('Justin Verlander')
prec_scores.append(jv_score)
adj_scores.append(jv_score/(1/3))

In [224]:
playerid_lookup('hendricks','kyle')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,hendricks,kyle,-1,NaN,NaN,-1,NaN,NaN
1,hendricks,kyle,543294,hendk001,hendrky01,12049,2014.0,2020.0


In [225]:
kh_score = pitch_classification(543294, 'score')

Gathering Player Data


In [257]:
names.append('Kyle Hendricks')
prec_scores.append(kh_score)
adj_scores.append(kh_score/0.25)

In [229]:
playerid_lookup('berrios','jose')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,berrios,jose,621244,berrj001,berrijo01,14168,2016.0,2020.0


In [230]:
jb_score = pitch_classification(621244, 'score')

Gathering Player Data


In [258]:
names.append('Jose Berrios')
prec_scores.append(jb_score)
adj_scores.append(jb_score/0.25)

In [234]:
playerid_lookup('odorizzi','jake')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,odorizzi,jake,543606,odorj001,odorija01,6397,2012.0,2019.0


In [235]:
jo_score = pitch_classification(543606, 'score')

Gathering Player Data


In [259]:
names.append('Jake Odorizzi')
prec_scores.append(jo_score)
adj_scores.append(jo_score/(1/3))

In [247]:
playerid_lookup('kershaw','clayton')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,kershaw,clayton,477132,kersc001,kershcl01,2036,2008.0,2019.0


In [260]:
ck_score = pitch_classification(477132, 'score')

Gathering Player Data


In [261]:
names.append('Clayton Kershaw')
prec_scores.append(ck_score)
adj_scores.append(ck_score/(1/3))

In [265]:
playerid_lookup('paxton','james')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,paxton,james,572020,paxtj001,paxtoja01,11828,2013.0,2020.0


In [266]:
jp_score = pitch_classification(572020,'score')

Gathering Player Data


In [267]:
names.append('James Paxton')
prec_scores.append(jp_score)
adj_scores.append(jp_score/(1/3))

In [272]:
playerid_lookup('snell','blake')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,snell,blake,605483,snelb001,snellbl01,13543,2016.0,2020.0


In [273]:
bs_score = pitch_classification(605483,'score')

Gathering Player Data


In [275]:
names.append('Blake Snell')
prec_scores.append(bs_score)
adj_scores.append(bs_score/(1/3))

In [282]:
playerid_lookup('nola','aaron')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,nola,aaron,605400,nolaa001,nolaaa01,16149,2015.0,2020.0


In [283]:
an_score = pitch_classification(605400,'score')

Gathering Player Data


In [284]:
names.append('Aaron Nola')
prec_scores.append(an_score)
adj_scores.append(an_score/0.25)

In [294]:
playerid_lookup('flaherty','jack')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,flaherty,jack,-1,NaN,NaN,-1,NaN,NaN
1,flaherty,jack,-1,NaN,NaN,-1,NaN,NaN
2,flaherty,jack,656427,flahj002,flaheja01,17479,2017.0,2020.0


In [295]:
jf_score = pitch_classification(656427, 'score')

Gathering Player Data


In [296]:
names.append('Jack Flaherty')
prec_scores.append(jf_score)
adj_scores.append(jf_score/0.25)

In [297]:
playerid_lookup('buehler','walker')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,buehler,walker,621111,buehw001,buehlwa01,19374,2017.0,2020.0


In [298]:
wb_score = pitch_classification(621111,'score')

Gathering Player Data


In [300]:

prec_scores.append(wb_score)
adj_scores.append(wb_score/0.25)

In [301]:
names

['Max Scherzer',
 'Gerrit Cole',
 'Tyler Glasnow',
 'Madison Bumgarner',
 'Chris Paddack',
 'Jacob DeGrom',
 'Justin Verlander',
 'Kyle Hendricks',
 'Jose Berrios',
 'Jake Odorizzi',
 'Clayton Kershaw',
 'James Paxton',
 'Blake Snell',
 'Aaron Nola',
 'Jack Flaherty',
 'Walker Buehler']

In [302]:
data = {'Name':names, 'Precision Score':prec_scores, 'Adjusted Score':adj_scores}

In [303]:
df = pd.DataFrame(data)

In [305]:
final_df = df.sort_values('Adjusted Score', ascending = False)

In [307]:
final_df = final_df.reset_index(drop=True)

In [308]:
final_df

,Name,Precision Score,Adjusted Score
0,Blake Snell,0.730963,2.192889
1,Jose Berrios,0.541334,2.165335
2,Kyle Hendricks,0.536563,2.146253
3,Aaron Nola,0.526152,2.104608
4,Clayton Kershaw,0.694630,2.083889
5,Chris Paddack,0.623755,1.871265
6,Max Scherzer,0.465156,1.860625
7,Walker Buehler,0.446816,1.787264
8,Jake Odorizzi,0.552780,1.658339
9,Justin Verlander,0.538281,1.614844
